In [31]:
import glob
import os
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

from monai.transforms import Resize, EnsureChannelFirst, LoadImage, Compose, ScaleIntensity, CenterSpatialCrop
from monai.data import ArrayDataset
from torch.utils.data import DataLoader


In [32]:
image_size = [512, 512]

train_image_path_remidio = "D:/Users/Michael/Documents/Uni/_Master/_Thesis/project/data/Chaksu/Train/1.0_Original_Fundus_Images/Remidio/"
train_segm_path_remidio = "D:/Users/Michael/Documents/Uni/_Master/_Thesis/project/data/Chaksu/Train/4.0_OD_CO_Fusion_Images/Remidio/STAPLE/"

transformer_remidio = Compose([LoadImage(image_only=True),
                               EnsureChannelFirst(),
                               ScaleIntensity(),
                               CenterSpatialCrop(roi_size=[2448, 2448]),
                               Resize(image_size)])

transformer_forus = Compose([LoadImage(image_only=True),
                             EnsureChannelFirst(),
                             ScaleIntensity(),
                             CenterSpatialCrop(roi_size=[1536, 1536]),
                             Resize(image_size)])

transformer_bosch = Compose([LoadImage(image_only=True),
                             EnsureChannelFirst(),
                             ScaleIntensity(),
                             CenterSpatialCrop(roi_size=[1440, 1440]),
                             Resize(image_size)])

data = ArrayDataset(img=sorted([train_image_path_remidio + file for file in os.listdir(train_image_path_remidio)]),
                    img_transform=transformer_remidio,
                    seg=sorted([train_segm_path_remidio + file for file in os.listdir(train_segm_path_remidio)]),
                    seg_transform=transformer_remidio)

dataloader = DataLoader(data,
                        batch_size=1,
                        shuffle=False,
                        num_workers=2)


In [33]:
new_path = "C:/Users/michi/Documents/Uni/_Master/_Thesis/project/data_chaksu/CHAKSU/"
new_path_img = new_path + "Images/"
new_path_segm = new_path + "Disc_Masks/"

new_index_increment = len(os.listdir(new_path_img))

for j, batch in tqdm(enumerate(dataloader)):
    temp = np.array(batch[1][0].permute(1, 2, 0) * 25).astype(np.uint8)
    new_image = Image.fromarray(np.array(batch[0][0].permute(1, 2, 0) * 255).astype(np.uint8))
    new_image.save(new_path_img + str(j + new_index_increment) + ".png")
    new_image = Image.fromarray(np.array(batch[1][0].permute(1, 2, 0) * 255).squeeze().astype(np.uint8))
    new_image.save(new_path_segm + str(j + new_index_increment) + ".png")

0it [00:00, ?it/s]